# Introduction to _py2cytoscape_
#### Pythonista-friendly wrapper for cyREST

### What is this?
From version 0.4.0, ___py2cytoscape___ has wrapper for [cyREST](http://apps.cytoscape.org/apps/cyrest) RESTful API.  This means you can access Cytoscape features in more _Pythonic_ way instead of calling raw REST API via HTTP.  Since [pandas](http://pandas.pydata.org/) is a standard library for data mangling/analysis in Python, this new version uses its [DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html#pandas.DataFrame) as its basic data object.

#### Questions or Feature Request
Send me an email (kono at ucsd).

In [1]:
from py2cytoscape.data.cynetwork import CyNetwork
from py2cytoscape.data.cyrest_client import CyRestClient
import py2cytoscape.util.cytoscapejs as cyjs

import networkx as nx
import pandas as pd

# Create an instance of cyREST client.  Default IP is 'localhost', and port number is 1234.
# cy = CyRestClient() - This default constructor creates connection to http://localhost:1234/v1
cy = CyRestClient(ip='127.0.0.1', port=1234)

# Cleanup
cy.network.delete_all()

## Creating empty networks
Creation will be done by the cyREST client.

In [2]:
# Empty network
empty1 = cy.network.create()

# With name
empty2 = cy.network.create(name='Created in Jupyter Notebook')

# With name and collection name
empty3 = cy.network.create(name='Also created in Jupyter', collection='New network collection')

## Create networks from various types of data
Currently, py2cytoscape accepts the following data as input:

* Cytoscape.js
* NetworkX
* Pandas DataFrame


* igraph (TBD)
* Numpy adjacency matrix (binary or weighted) (TBD)
* GraphX (TBD)

In [3]:
# Cytoscape.js JSON
n1 = cy.network.create(data=cyjs.get_empty_network(), name='Created from Cytoscape.js JSON')

# Pandas DataFrame
df_from_sif = pd.read_csv('../tests/data/galFiltered.sif', names=['source', 'interaction', 'target'], sep=' ')
yeast1 = cy.network.create_from_dataframe(df_from_sif, name='Yeast network created from pandas DataFrame')

# NetworkX
nx_graph  = nx.scale_free_graph(100)
nx.set_node_attributes(nx_graph, 'Degree', nx.degree(nx_graph))
nx.set_node_attributes(nx_graph, 'Betweenness_Centrality', nx.betweenness_centrality(nx_graph))
scale_free100 = cy.network.create_from_networkx(nx_graph, collection='Generated by NetworkX')

# TODO: igraph
# TODO: Numpy adj. martix
# TODO: GraphX

## Get Network from Cytoscape

You can get network data in the following forms:

* Cytoscape.js
* NetworkX
* DataFrame

In [4]:
yeast1_json = yeast1.to_json()

yeast1_nx = yeast1.to_networkx()

yeast1_df = yeast1.to_dataframe()
yeast1_df.head()

,source,interaction,target
0,YDR277C,pp,YDL194W
1,YDR277C,pp,YJR022W
2,YPR145W,pp,YMR117C
3,YER054C,pp,YBR045C
4,YER054C,pp,YER133W


## Working with CyNetwork API
___CyNetwork___ class is a simple wrapper to cyREST raw REST API.  __It does not hold the actual network data.  It's a reference to a network in current Cytoscape session__.  With CyNetwork API, you can access Cytoscape data objects in more Pythonista-friendly way.

In [5]:
network_suid = yeast1.get_id()
print('This object references to Cytoscape network with SUID ' + str(network_suid) + '\n')
print('And its name is: ' + yeast1.get_network_value(column='name') + '\n')

nodes = yeast1.get_nodes()
edges = yeast1.get_edges()

print('* This network has ' + str(len(nodes)) + ' nodes and ' + str(len(edges)) + ' edges\n') 

# Get a row in the node table as pandas Series object
node0 = nodes[0]
row = yeast1.get_node_value(id=node0)
print(row)

# Or, pick one cell in the table
cell = yeast1.get_node_value(id=node0, column='name')
print('\nThis node has name: ' + cell) 

This object references to Cytoscape network with SUID 103415

And its name is: Yeast network created from pandas DataFrame

* This network has 331 nodes and 362 edges

SUID            103425
id             YKR026C
name           YKR026C
selected         False
shared_name    YKR026C
dtype: object

This node has name: YKR026C


### Get references to existing networks
And of course, you can grab references to existing Cytoscape networks:

In [6]:
# Create a new CyNetwork object from existing network
network_ref1 = cy.network.create(suid=yeast1.get_id())

# And they are considered as same objects.
print(network_ref1 == yeast1)
print(network_ref1.get_network_value(column='name'))

True
Yeast network created from pandas DataFrame


## Tables as DataFrame
Cytoscape has two main data types: ___Network___ and ___Table___.  Network is the graph topology, and Tables are properties for those graphs.  For simplicity, this library has access to three basic table objects:

* Node Table
* Edge Table
* Network Table

For 99% of your use cases, you can use these three to store properties.  Since [pandas](http://pandas.pydata.org/) is extremely useful to handle table data, default data type for tables is __DataFrame__.  However, you can also use other data types including:

* Cytoscape.js style JSON
* CSV
* TSV
* CX (TBD)

In [7]:
# Get table  from Cytoscape
node_table = scale_free100.get_node_table()
edge_table = scale_free100.get_edge_table()
network_table = scale_free100.get_network_table()

In [8]:
node_table.head()

,shared name,name,selected,id,Degree,Betweenness_Centrality
SUID,,,,,,
104839,0,0,False,0,85,0.099138
104840,1,1,False,1,68,0.039597
104841,2,2,False,2,56,0.040370
104842,3,3,False,3,9,0.000000
104843,4,4,False,4,11,0.043393


In [9]:
network_table.transpose().head()

SUID,104829
shared name,"directed_scale_free_graph(100,alpha=0.41,beta=..."
name,"directed_scale_free_graph(100,alpha=0.41,beta=..."
selected,True
__Annotations,NaN


In [10]:
names = scale_free100.get_node_column('Degree')
print(names)

# Node Column information.  "name" is the unique Index
scale_free100.get_node_columns()

name                                                 Degree
values    [85.0, 56.0, 68.0, 11.0, 9.0, 22.0, 4.0, 12.0,...
dtype: object


,immutable,primaryKey,type
name,,,
SUID,True,True,Long
shared name,True,False,String
name,True,False,String
selected,True,False,Boolean
id,False,False,String
Degree,False,False,Double
Betweenness_Centrality,False,False,Double


## Network Topology Modification

### Adding and deleteing nodes/edges

In [11]:
# Add new nodes: Simply send the list of node names.  NAMES SHOULD BE UNIQUE!
new_node_names = ['a', 'b', 'c']
# Return value contains dictionary from name to SUID.
new_nodes = scale_free100.add_nodes(new_node_names)

# Add new edges
# Send a list of tuples:  (source node SUID, target node SUID, interaction type
new_edges = []
new_edges.append((new_nodes['a'], new_nodes['b'], 'type1'))
new_edges.append((new_nodes['a'], new_nodes['c'], 'type2'))
new_edges.append((new_nodes['b'], new_nodes['c'], 'type3'))

new_edge_ids = scale_free100.add_edges(new_edges)
new_edge_ids

,source,target
SUID,,
105497,105491,105492
105498,105491,105493
105499,105492,105493


In [12]:
# Delete node
scale_free100.delete_node(new_nodes['a'])

# Delete edge
scale_free100.delete_edge(new_edge_ids.index[0])

## Update Table
Let's do something a bit more realistic.  You can update any Tables by using DataFrame objects.

### 1. ID conversion with external service
Let's use [ID Conversion web service by Uniprot](http://www.uniprot.org/help/programmatic_access) to add more information to existing yeast network in current session.

In [13]:
# Small utility function to convert ID sets
import requests
import StringIO

def uniprot_id_mapping_service(query=None, from_id=None, to_id=None):
    # Uniprot ID Mapping service
    url = 'http://www.uniprot.org/mapping/'
    payload = {
        'from': from_id,
        'to': to_id,
        'format':'tab',
        'query': query
    }
    res = StringIO.StringIO(requests.get(url, params=payload).content)
    df = pd.read_csv(res, sep='\t')
    res.close()
    return df

In [14]:
# Get node table from Cytoscape
yeast_node_table = yeast1.get_node_table()

# From KEGG ID to UniprotKB ID
query1 = ' '.join(yeast_node_table['name'].map(lambda gene_id: 'sce:' + gene_id).values)
id_map_kegg2uniprot = uniprot_id_mapping_service(query1, from_id='KEGG_ID', to_id='ID')
id_map_kegg2uniprot.columns = ['kegg', 'uniprot']

# From UniprotKB to SGD
query2 = ' '.join(id_map_kegg2uniprot['uniprot'].values)
id_map_uniprot2sgd = uniprot_id_mapping_service(query2, from_id='ID', to_id='SGD_ID')
id_map_uniprot2sgd.columns = ['uniprot', 'sgd']

# From UniprotKB to Entrez Gene ID
query3 = ' '.join(id_map_kegg2uniprot['uniprot'].values)
id_map_uniprot2ncbi = uniprot_id_mapping_service(query3, from_id='ID', to_id='P_ENTREZGENEID')
id_map_uniprot2ncbi.columns = ['uniprot', 'entrez']

# Merge them
merged = pd.merge(id_map_kegg2uniprot, id_map_uniprot2sgd, on='uniprot')
merged = pd.merge(merged, id_map_uniprot2ncbi, on='uniprot')

# Add key column by removing prefix
merged['name'] = merged['kegg'].map(lambda kegg_id : kegg_id[4:])
merged.head()

,kegg,uniprot,sgd,entrez,name
0,sce:YKR026C,EI2BA_YEAST,S000001734,853896,YKR026C
1,sce:YGL122C,NAB2_YEAST,S000003090,852755,YGL122C
2,sce:YGR218W,XPO1_YEAST,S000003450,853133,YGR218W
3,sce:YGL097W,RCC1_YEAST,S000003065,852782,YGL097W
4,sce:YOR204W,DED1_YEAST,S000005730,854379,YOR204W


In [15]:
# Now update existing node table with the data frame above.
yeast1.update_node_table(merged, network_key_col='name', data_key_col='name')

# Check the table is actually updated
yeast1.get_node_table().head()

,shared name,name,selected,id,kegg,entrez,sgd,uniprot
SUID,,,,,,,,
103425,YKR026C,YKR026C,False,YKR026C,sce:YKR026C,853896,S000001734,EI2BA_YEAST
103426,YGL122C,YGL122C,False,YGL122C,sce:YGL122C,852755,S000003090,NAB2_YEAST
103427,YGR218W,YGR218W,False,YGR218W,sce:YGR218W,853133,S000003450,XPO1_YEAST
103428,YGL097W,YGL097W,False,YGL097W,sce:YGL097W,852782,S000003065,RCC1_YEAST
103429,YOR204W,YOR204W,False,YOR204W,sce:YOR204W,854379,S000005730,DED1_YEAST


## Create / Delete Table Data
Currently, ___you cannot delete the table or rows___ due to the Cytoscape data model design.  However, it is easy to create / delete columns:

In [16]:
# Delete columns
yeast1.delete_node_table_column('kegg')

# Create columns
yeast1.create_node_column(name='New Empty Double Column', data_type='Double', is_immutable=False, is_list=False)

# Default is String, mutable column.
yeast1.create_node_column(name='Empty String Col')

yeast1.get_node_table().head()

,shared name,name,selected,id,entrez,sgd,uniprot,New Empty Double Column,Empty String Col
SUID,,,,,,,,,
103425,YKR026C,YKR026C,False,YKR026C,853896,S000001734,EI2BA_YEAST,NaN,NaN
103426,YGL122C,YGL122C,False,YGL122C,852755,S000003090,NAB2_YEAST,NaN,NaN
103427,YGR218W,YGR218W,False,YGR218W,853133,S000003450,XPO1_YEAST,NaN,NaN
103428,YGL097W,YGL097W,False,YGL097W,852782,S000003065,RCC1_YEAST,NaN,NaN
103429,YOR204W,YOR204W,False,YOR204W,854379,S000005730,DED1_YEAST,NaN,NaN


## Visual Styles
You can also use wrapper API to access Visual Styles.

Current limitations are:

* You need to use unique name for the Styles
* Need to know how to write serialized form of objects

In [17]:
# Get all existing Visual Styles
import json
styles = cy.style.get_all()
print(json.dumps(styles, indent=4))

# Create a new style
style1 = cy.style.create('sample_style1')

# Get a reference to the existing style
default_style = cy.style.create('default')

print(style1.get_name())
print(default_style.get_name())

# Get all available Visual Properties
print(len(cy.style.vps.get_all()))

# Get  Visual Properties for each data type
node_vps = cy.style.vps.get_node_visual_props()
edge_vps = cy.style.vps.get_edge_visual_props()
network_vps = cy.style.vps.get_network_visual_props()

pd.Series(edge_vps)

[
    "Universe", 
    "Big Labels", 
    "default", 
    "Directed", 
    "Ripple", 
    "sample_style1", 
    "Nested Network Style", 
    "Sample1", 
    "default black", 
    "Solid", 
    "Minimal"
]
sample_style1
default
103


0                                   EDGE
1                              EDGE_BEND
2                            EDGE_CURVED
3                             EDGE_LABEL
4                       EDGE_LABEL_COLOR
5                   EDGE_LABEL_FONT_FACE
6                   EDGE_LABEL_FONT_SIZE
7                EDGE_LABEL_TRANSPARENCY
8                         EDGE_LINE_TYPE
9                             EDGE_PAINT
10                         EDGE_SELECTED
11                   EDGE_SELECTED_PAINT
12      EDGE_SOURCE_ARROW_SELECTED_PAINT
13               EDGE_SOURCE_ARROW_SHAPE
14    EDGE_SOURCE_ARROW_UNSELECTED_PAINT
15            EDGE_STROKE_SELECTED_PAINT
16          EDGE_STROKE_UNSELECTED_PAINT
17      EDGE_TARGET_ARROW_SELECTED_PAINT
18               EDGE_TARGET_ARROW_SHAPE
19    EDGE_TARGET_ARROW_UNSELECTED_PAINT
20                          EDGE_TOOLTIP
21                     EDGE_TRANSPARENCY
22                 EDGE_UNSELECTED_PAINT
23                          EDGE_VISIBLE
24              

### Set default values
To set default values for Visual Properties, simply pass key-value pairs as dictionary.

In [18]:
# Prepare key-value pair for defaults
new_defaults = {
    # Node defaults
    'NODE_FILL_COLOR': '#eeeeff',
    'NODE_SIZE': 20,
    'NODE_BORDER_WIDTH': 0,
    'NODE_TRANSPARENCY': 120,
    'NODE_LABEL_COLOR': 'white',
    
    # Edge defaults
    'EDGE_WIDTH': 3,
    'EDGE_STROKE_UNSELECTED_PAINT': '#aaaaaa',
    'EDGE_LINE_TYPE': 'LONG_DASH',
    'EDGE_TRANSPARENCY': 120,
    
    # Network defaults
    'NETWORK_BACKGROUND_PAINT': 'black'
}

# Update
style1.update_defaults(new_defaults)

# Apply the new style
cy.style.apply(style1, yeast1)

### Mappings
(TBD)

In [19]:
# Passthrough mapping
style1.create_passthrough_mapping(column='name', dataType='String', vp='NODE_LABEL')

# Discrete mapping
kv_pair = {
    'pp': 'pink',
    'pd': 'green'
}

style1.create_discrete_mapping(column='interaction', 
                               dataType='String', vp='EDGE_STROKE_UNSELECTED_PAINT', mappings=kv_pair)

# Continuous mapping
points = [
    {
        'value': '1.0',
        'lesser':'white',
        'equal':'white',
        'greater': 'white'
    },
    {
        'value': '20.0',
        'lesser':'red',
        'equal':'red',
        'greater': 'red'
    }
]

minimal_style = cy.style.create('Minimal')
minimal_style.create_continuous_mapping(column='Degree', dataType='Double', vp='NODE_FILL_COLOR', points=points)

# Apply the new style
cy.style.apply(minimal_style, scale_free100)

## Layouts
Currently, this supports automatic layouts with default parameters.

In [20]:
import json
layouts = cy.layout.get_all()
print(json.dumps(layouts, indent=4))

[
    "attribute-circle", 
    "stacked-node-layout", 
    "degree-circle", 
    "circular", 
    "attributes-layout", 
    "kamada-kawai", 
    "force-directed", 
    "grid", 
    "hierarchical", 
    "fruchterman-rheingold", 
    "isom"
]


In [21]:
cy.layout.apply(name='circular', network=yeast1)
yeast1.get_views()
yeast_view1 = yeast1.get_first_view()
node_views = yeast_view1['elements']['nodes']
df3 = pd.DataFrame(node_views)
df3.head()

,data,position,selected
0,"{u'name': u'YDL194W', u'SUID': 103755, u'share...","{u'y': 337.340804913, u'x': -816.208036881}",False
1,"{u'name': u'YDR277C', u'SUID': 103754, u'share...","{u'y': 283.547044268, u'x': -768.351675593}",False
2,"{u'name': u'YBR043C', u'SUID': 103753, u'share...","{u'y': 700.01224468, u'x': 166.447801031}",False
3,"{u'name': u'YPR145W', u'SUID': 103752, u'share...","{u'y': 549.176777084, u'x': -524.657171636}",False
4,"{u'name': u'YER054C', u'SUID': 103751, u'share...","{u'y': -104.597393513, u'x': 1446.50457128}",False


## Embed Interactive Widget

In [22]:
from py2cytoscape.cytoscapejs import viewer as cyjs
cy.layout.apply(network=scale_free100)
view1 = scale_free100.get_first_view()
view2 = yeast1.get_first_view()
# print(view1)
cyjs.render(view2, 'default2', background='#efefef')

<IPython.core.display.Javascript object>

In [23]:
scale_free100.get_node_columns().ix['name']['type']
df = style1.get_defaults()
df2 = style1.get_mappings()
df.head()

[{u'visualProperty': u'EDGE_BEND', u'value': None}, {u'visualProperty': u'EDGE_CURVED', u'value': True}, {u'visualProperty': u'EDGE_LABEL', u'value': u''}, {u'visualProperty': u'EDGE_LABEL_COLOR', u'value': u'#000000'}, {u'visualProperty': u'EDGE_LABEL_FONT_FACE', u'value': u'SansSerif,plain,10'}, {u'visualProperty': u'EDGE_LABEL_FONT_SIZE', u'value': 10}, {u'visualProperty': u'EDGE_LABEL_TRANSPARENCY', u'value': 255}, {u'visualProperty': u'EDGE_LINE_TYPE', u'value': u'LONG_DASH'}, {u'visualProperty': u'EDGE_SELECTED', u'value': False}, {u'visualProperty': u'EDGE_SOURCE_ARROW_SELECTED_PAINT', u'value': u'#FFFF00'}, {u'visualProperty': u'EDGE_SOURCE_ARROW_SHAPE', u'value': u'NONE'}, {u'visualProperty': u'EDGE_SOURCE_ARROW_UNSELECTED_PAINT', u'value': u'#000000'}, {u'visualProperty': u'EDGE_STROKE_SELECTED_PAINT', u'value': u'#FF0000'}, {u'visualProperty': u'EDGE_STROKE_UNSELECTED_PAINT', u'value': u'#AAAAAA'}, {u'visualProperty': u'EDGE_TARGET_ARROW_SELECTED_PAINT', u'value': u'#FFFF00'

EDGE_BEND                             None
EDGE_CURVED                           True
EDGE_LABEL                                
EDGE_LABEL_COLOR                   #000000
EDGE_LABEL_FONT_FACE    SansSerif,plain,10
dtype: object